In [1]:
# Import required libraries
#!pip install pandas dash
#!pip install pandas
#!pip install dash
#!pip install notebook

#!pip install pandas (Had to install this first on my PC/Environment)
#!pip install dash (Had to install this first on my PC/Environment)
#!pip install plotly (This initially installed my Pandas LIbrary which previously didn't exist on my PC) *This requires pandas to be installed
#PLOTLY POSSIBLE NOTE: For Jupyter Lab you need jupyterlab-plotly
import pandas as pd
import dash
from dash import html #Original code of 'dash_html_components as html' was deprecated. Replaced with 'from dash import html'.
from dash import dcc   #Original code of 'import dash_core_components as dcc'. Replaced with 'from dash import html'.
from dash.dependencies import Input, Output
import plotly.express as px

print(pd.__version__) #get the pandas version
print(dash.__version__) #get the plotly dash version

1.5.3
2.9.1


In [2]:
# Read the launch data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                         {'label': 'ALL SITES', 'value': 'ALL'},
                                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                    ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here", 
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,max=10000,step=1000,
                                                value=[min_payload,max_payload],
                                                marks={0: '0', 2500:'2500',5000:'5000',
                                                7500:'7500', 10000: '10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def build_graph(site_dropdown):
    if site_dropdown == 'ALL':
        piechart = px.pie(data_frame = spacex_df, names='Launch Site', values='class' ,title='Total Successful Launches for All Sites')
        return piechart
    else:
        #specific_df = spacex_df['Launch Site']
        specific_df=spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        piechart = px.pie(data_frame = specific_df, names='class',title='Total Launch for a Specific Site')
        return piechart

In [ ]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')])

def update_graph(site_dropdown, payload_slider):
    if site_dropdown == 'ALL':
        filtered_data = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0])
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="Payload Mass (kg)", y="class", 
        color="Booster Version Category",title="Correlation Between Payload And Success (All Sites)")
        return scatterplot
    else:
        specific_df=spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        filtered_data = specific_df[(specific_df['Payload Mass (kg)']>=payload_slider[0])
        &(spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        scatterplot = px.scatter(data_frame=filtered_data, x="Payload Mass (kg)", y="class", 
        color="Booster Version Category",title="Correlation Between Payload And Success (All Sites)")
        return scatterplot

# Run the app
if __name__ == '__main__':
    app.run_server()
    
# It is running here: http://127.0.0.1:8050 (put this in my local Browser)

#1. Which site has the largest successful launches? (KSC LC-39A)

#2. Which site has the highest launch success rate? (KSC LC-39A)

#3. Which payload range(s) has the highest launch success rate? (2500kg-5000kg)

#3supp. Details.....Payload Range (0-2500). Success=(7) + Failure=(12) = 19 (Success Rate=36.8%)
#3supp. Details.....#Payload Range (2500-5000). Success=(11) + Failure=(9) = 20  (Success Rate=55%)
#3supp. Details.....#Payload Range (5000-7500). Success=(2) + Failure=(7) = 9 (Success Rate=22.2%)
#3supp. Details.....#Payload Range (7500-10000). Success=(1) + Failure=(1) = 2 (Success Rate=50%)

#4. Which payload range(s) has the lowest launch success rate? (5000kg-7500kg)

#4supp. Details.....Payload Range (0-2500). Success=(7) + Failure=(12) = 19 (Success Rate=36.8%)
#4supp. Details.....#Payload Range (2500-5000). Success=(11) + Failure=(9) = 20  (Success Rate=55%)
#4supp. Details.....#Payload Range (5000-7500). Success=(2) + Failure=(7) = 9 (Success Rate=22.2%)
#4supp. Details.....#Payload Range (7500-10000). Success=(1) + Failure=(1) = 2 (Success Rate=50%)

#5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

#5supp. Details.....v1.0. Success=(0) + Failure=(4) = 4 (Success Rate=0%)
#5supp. Details.....v1.1. Success=(1) + Failure=(14) = 15 (Success Rate=7.1%)
#5supp. Details.....FT. Success=(15) + Failure=(8) = 23 (Success Rate=65.2%)
#5supp. Details.....B4. Success=(6) + Failure=(5) = 11 (Success Rate-54.5%)
#5supp. Details.....B5. Success=(1) + Failure=(0) = 1 (Success Rate-100%)

    